In [444]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [445]:
cd drive/MyDrive/NLP

[Errno 2] No such file or directory: 'drive/MyDrive/NLP'
/content/drive/MyDrive/NLP


In [446]:
import pandas as pd
import numpy as np
# import torch
from sklearn.model_selection import train_test_split

In [447]:
df = pd.read_csv('AlexaCombinedReviews.csv')
df.head()

,verified_reviews,paraphrased_reviews
0,Love my Echo!,\n\nI'm so fond of my Echo!
1,Loved it!,\n\nI adored it!
2,"Sometimes while playing a game, you can answer...","\n\nSometimes when playing a game, I can answe..."
3,I have had a lot of fun with this thing. My 4 ...,\n\nI've had a blast with this device. My four...
4,Music,is the food of the soul\n\nMusic nourishes th...


In [448]:
df.shape

(3150, 2)

In [449]:
df.isnull().sum()

verified_reviews       0
paraphrased_reviews    1
dtype: int64

In [450]:
mask = df.isnull()
row_has_missing = mask.any(axis=1)
print("Row indices with missing values:")
print(df.index[row_has_missing].tolist())

Row indices with missing values:
[434]


In [451]:
df.dropna(inplace=True)

In [452]:
df.shape

(3149, 2)

In [453]:
import re
# define a regular expression to match any newline character
newline_regex = re.compile(r'\r?\n')

# define a lambda function to remove any newline characters from a string
remove_newlines = lambda s: newline_regex.sub('', s)

# apply the lambda function to the 'paraphrased_reviews' column
df['paraphrased_reviews'] = df['paraphrased_reviews'].apply(remove_newlines)

In [454]:
# for i in df['paraphrased_reviews']:
#     if '\n' in i:
#        i=i.replace('\n','')

In [455]:
# def remove_char(remove_newlines):
#     remove_newlines = lambda x: x.replace('\n', '')
#     return remove_newlines

In [456]:
# df['paraphrased_reviews'] = df['paraphrased_reviews'].replace(remove_char)

In [457]:
df.head()

,verified_reviews,paraphrased_reviews
0,Love my Echo!,I'm so fond of my Echo!
1,Loved it!,I adored it!
2,"Sometimes while playing a game, you can answer...","Sometimes when playing a game, I can answer a ..."
3,I have had a lot of fun with this thing. My 4 ...,I've had a blast with this device. My four yea...
4,Music,is the food of the soulMusic nourishes the sp...


In [458]:
count=0
for i in df['verified_reviews']:
    if len(i)==1:
        count+=1
        # print(i)
print(count)



81


In [459]:
df = df[~(df['verified_reviews'].str.len() == 1)]

In [460]:
df.head()

,verified_reviews,paraphrased_reviews
0,Love my Echo!,I'm so fond of my Echo!
1,Loved it!,I adored it!
2,"Sometimes while playing a game, you can answer...","Sometimes when playing a game, I can answer a ..."
3,I have had a lot of fun with this thing. My 4 ...,I've had a blast with this device. My four yea...
4,Music,is the food of the soulMusic nourishes the sp...


In [461]:
df['verified_reviews'].isnull().sum()

0

In [462]:
df = df[~(df['verified_reviews'].str.split(" ").str.len() == 1)]

In [463]:
df.shape

(2997, 2)

In [464]:
df.tail()

,verified_reviews,paraphrased_reviews
3144,love it,or leave itTake it or leave it.
3145,"Perfect for kids, adults and everyone in betwe...",Great for all ages!
3146,"Listening to music, searching locations, check...","There are many more tasks that can be learned,..."
3147,"I do love these things, i have them running my...",I'm really fond of these gadgets; they control...
3148,Only complaint I have is that the sound qualit...,My only issue with the dot is the sound qualit...


In [465]:
emoticons = re.compile('[:;=]-?[)(DP]')

df['verified_reviews'] = df['verified_reviews'].replace(emoticons, '')

# print(df["verified_reviews"][19])

In [466]:
# from transformers import GPT2Tokenizer
# model_name="gpt2"


In [467]:
dataframe = pd.DataFrame()
for row in df.itertuples():
  dataframe = pd.concat([dataframe, pd.DataFrame.from_records([{ 'reviews': row.verified_reviews, "isHuman": 1}])], ignore_index=True)
  dataframe = pd.concat([dataframe, pd.DataFrame.from_records([{ 'reviews': row.paraphrased_reviews, "isHuman": 0}])], ignore_index=True)
dataframe.head()

,reviews,isHuman
0,Love my Echo!,1
1,I'm so fond of my Echo!,0
2,Loved it!,1
3,I adored it!,0
4,"Sometimes while playing a game, you can answer...",1


In [468]:
Y = dataframe["isHuman"]
X = dataframe.drop('isHuman',axis=1)

In [469]:
# pip install transformers

In [470]:
from transformers import AutoTokenizer, GPT2Config, GPT2Model
import torch
from progressbar import progressbar
from csv import writer


tokenizer = AutoTokenizer.from_pretrained('gpt2')
# dataframe['reviews'] = dataframe['reviews'].apply(lambda x: tokenizer(x, return_tensors='pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GPT2Model.from_pretrained('gpt2')
# torch.device(device)
# dataframe['reviews'] = dataframe['reviews'].apply(lambda x: model(x))

# Define a function to embed a sentence using GPT-2
def embed_sentence_gpt2(sentence):
  # print(sentence)
  # Tokenize the sentence
  encoded_input = tokenizer(sentence, return_tensors='pt')
  # Get the model's hidden state for the sentence
  with torch.no_grad():
    output = model(**encoded_input)
  embeddings = output.last_hidden_state.mean(dim=1).numpy()
  return embeddings

# # Example usage:
# # sentence = "This is a test sentence."
# # gpt2_embedding = embed_sentence_gpt2(sentence)
# # df['verified_reviews'] = df['verified_reviews'] + tokenizer.eos_token
# # df['verified_reviews'][0]

# print(df)
tempList = []
dfTemp = pd.DataFrame()
with open('embedding_gen.csv', 'a') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(["embedding"])
    f_object.close()
# print(tempList)
for review in progressbar(dataframe['reviews']):
  embedding = embed_sentence_gpt2(review)
  with open('embedding_gen.csv', 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(embedding)
        f_object.close()

tempList


# Tokenize text column
# dataframe['reviews'] = dataframe['reviews'].apply(lambda x: embed_sentence_gpt2(x))
# df['paraphrased_reviews'] = df['paraphrased_reviews'].apply(lambda x: embed_sentence_gpt2(x))

# # Print updated data frame
# print(df)


cuda


100% (5994 of 5994) |####################| Elapsed Time: 0:16:46 Time:  0:16:46


[]

In [471]:
x_train, y_train,x_test,y_test = train_test_split(df['reviews'], df["isHuman"], test_size=0.2, random_state=265)
print(x_train)

KeyError: ignored

In [ ]:
c = 1048576
print(0x80 | (c >> 21))
# print(0x80 | (c & 0x3F))
# print(0xe0 | (c >> 18))
# print(0x80 | (c & 0x3F))